In [1]:
!ls /kaggle/input/kitti360-3d-semantics/data_3d_semantics/train/2013_05_28_drive_0007_sync/dynamic

0000000002_0000000125.ply  0000000785_0000000870.ply  0000001659_0000001750.ply
0000000119_0000000213.ply  0000000865_0000000952.ply  0000001745_0000001847.ply
0000000208_0000000298.ply  0000000947_0000001039.ply  0000001841_0000001957.ply
0000000293_0000000383.ply  0000001034_0000001127.ply  0000001950_0000002251.ply
0000000378_0000000466.ply  0000001122_0000001227.ply  0000002237_0000002410.ply
0000000461_0000000547.ply  0000001221_0000001348.ply  0000002395_0000002789.ply
0000000542_0000000629.ply  0000001340_0000001490.ply  0000002782_0000002902.ply
0000000624_0000000710.ply  0000001483_0000001582.ply
0000000705_0000000790.ply  0000001577_0000001664.ply


In [2]:
!ls /kaggle/input/kitti360-3d-semantics/data_3d_semantics/train/2013_05_28_drive_0000_sync/dynamic

0000000002_0000000385.ply  0000003919_0000004105.ply  0000007777_0000007982.ply
0000000372_0000000610.ply  0000004093_0000004408.ply  0000007968_0000008291.ply
0000000599_0000000846.ply  0000004397_0000004645.ply  0000008278_0000008507.ply
0000000834_0000001286.ply  0000004631_0000004927.ply  0000008496_0000008790.ply
0000001270_0000001549.ply  0000004916_0000005264.ply  0000008779_0000009015.ply
0000001537_0000001755.ply  0000005249_0000005900.ply  0000009003_0000009677.ply
0000001740_0000001991.ply  0000005880_0000006165.ply  0000009666_0000009895.ply
0000001980_0000002295.ply  0000006154_0000006400.ply  0000009886_0000010098.ply
0000002282_0000002514.ply  0000006387_0000006634.ply  0000010078_0000010362.ply
0000002501_0000002706.ply  0000006623_0000006851.ply  0000010352_0000010588.ply
0000002695_0000002925.ply  0000006828_0000007055.ply  0000010577_0000010841.ply
0000002913_0000003233.ply  0000007044_0000007286.ply  0000010830_0000011124.ply
0000003221_0000003475.ply  0000007277_00

In [3]:
!pip3 install open3d

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.7/447.7 MB 3.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 93.2 MB/s eta 0:00:00:00:0100:01


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import open3d as o3d
from torch.utils.data import Dataset, DataLoader
import numpy as np
import os
import glob
from scipy.spatial import cKDTree
import warnings
import torch.nn.functional as F


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [5]:
!ls /kaggle/input/kitti360-3d-semantics/data_3d_semantics/train

2013_05_28_drive_0000_sync  2013_05_28_drive_0007_sync
2013_05_28_drive_0002_sync  2013_05_28_drive_0009_sync
2013_05_28_drive_0003_sync  2013_05_28_drive_0010_sync
2013_05_28_drive_0004_sync  2013_05_28_drive_train.txt
2013_05_28_drive_0005_sync  2013_05_28_drive_val.txt
2013_05_28_drive_0006_sync


In [6]:
def collect_data(data_path, voxel_size=0.1):
    """
    data_path = '/kaggle/input/kitti360-3d-semantics/data_3d_semantics/train'
    """
    all_files = 0
    skipped_files = 0
    pairs = []
    source_dirs = sorted(os.listdir(data_path))#[:1]
    for dir in source_dirs:#[:1]
        source_path = os.path.join(data_path, dir)
        static_path = os.path.join(source_path, "static")
        dynamic_path = os.path.join(source_path, "dynamic")

        static_files = glob.glob(os.path.join(static_path, "*.ply"))
        for static in static_files:#[:10]:#[:1]
            name = os.path.basename(static)
            dynamic = os.path.join(dynamic_path, name)
            if (os.path.exists(dynamic)):
                print(static)
                all_files+=1
                # pairs.append((static, dynamic))
                static_pcd = o3d.io.read_point_cloud(static)
                dynamic_pcd = o3d.io.read_point_cloud(dynamic)
                if static_pcd.has_points() and dynamic_pcd.has_points():
                    # pairs.append((static, dynamic))
                    static_pcd = static_pcd.voxel_down_sample(voxel_size)
                    dynamic_pcd = dynamic_pcd.voxel_down_sample(voxel_size)

                    static_pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.2, max_nn=30))
                    dynamic_pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.2, max_nn=30))

                    static_normals = np.array(static_pcd.normals)
                    dynamic_normals = np.array(dynamic_pcd.normals)                    

                    static = np.array(static_pcd.points)
                    dynamic = np.array(dynamic_pcd.points)

                    if len(static) == 0 and len(dynamic) < 1:
                        return None

                    static_colors = np.asarray(static_pcd.colors) 
                    dynamic_colors = np.asarray(dynamic_pcd.colors)
                    # print(static_colors, dynamic_colors)

                    static = np.hstack([static, static_colors, static_normals])
                    dynamic = np.hstack([dynamic, dynamic_colors, dynamic_normals])
                    pairs.append((static, dynamic))
                else:
                    skipped_files+=1
                    print(f"Пропуск пустого: static={static}, dynamic={dynamic}")
    return pairs, all_files, skipped_files

pairs, all_files, skipped_files = collect_data('/kaggle/input/kitti360-3d-semantics/data_3d_semantics/train')
output_dir = '/kaggle/working/preprocessed_pairs'
os.makedirs(output_dir, exist_ok=True)

for i, (static, dynamic) in enumerate(pairs):
    np.savez(os.path.join(output_dir, f'pair_{i}.npz'), static=static, dynamic=dynamic)

print(f"Сохранено {len(pairs)} файлов в {output_dir}, пропущенно: {skipped_files}")

/kaggle/input/kitti360-3d-semantics/data_3d_semantics/train/2013_05_28_drive_0000_sync/static/0000009003_0000009677.ply
/kaggle/input/kitti360-3d-semantics/data_3d_semantics/train/2013_05_28_drive_0000_sync/static/0000002501_0000002706.ply
/kaggle/input/kitti360-3d-semantics/data_3d_semantics/train/2013_05_28_drive_0000_sync/static/0000003711_0000003928.ply
/kaggle/input/kitti360-3d-semantics/data_3d_semantics/train/2013_05_28_drive_0000_sync/static/0000004916_0000005264.ply
/kaggle/input/kitti360-3d-semantics/data_3d_semantics/train/2013_05_28_drive_0000_sync/static/0000010577_0000010841.ply
/kaggle/input/kitti360-3d-semantics/data_3d_semantics/train/2013_05_28_drive_0000_sync/static/0000007044_0000007286.ply
/kaggle/input/kitti360-3d-semantics/data_3d_semantics/train/2013_05_28_drive_0000_sync/static/0000009666_0000009895.ply
/kaggle/input/kitti360-3d-semantics/data_3d_semantics/train/2013_05_28_drive_0000_sync/static/0000008278_0000008507.ply
/kaggle/input/kitti360-3d-semantics/data

In [10]:
import json
import subprocess

KAGGLE_JSON_PATH = '/kaggle/input/kaggle/kaggle.json'  
!mkdir -p /root/.kaggle
!cp {KAGGLE_JSON_PATH} /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

In [11]:
dataset_metadata = {
    "title": "Preprocessed-KITTI360-Pairs",
    "id": "cvbnqq/kitti360-preprocessed-pairs",  
    "licenses": [{"name": "CC0-1.0"}],
    "description": "Preprocessed pairs (static and dynamic point clouds) from KITTI-360 semantics dataset"
}

metadata_path = os.path.join(output_dir, 'dataset-metadata.json')
with open(metadata_path, 'w') as f:
    json.dump(dataset_metadata, f)

In [ ]:
result = subprocess.run([
    'kaggle', 'datasets', 'create',
    '-p', output_dir, 
    '--dir-mode', 'zip' 
], capture_output=False, text=True)

print("Output:", result.stdout)
print("Errors:", result.stderr)

Starting upload for file pair_203.npz
Error while trying to load upload info: ApiStartBlobUploadRequest.__init__() got an unexpected keyword argument 'type'


100%|██████████| 52.0M/52.0M [00:02<00:00, 22.9MB/s]
  0%|          | 0.00/65.9M [00:00<?, ?B/s]

Upload successful: pair_203.npz (52MB)
Starting upload for file pair_140.npz
Error while trying to load upload info: ApiStartBlobUploadRequest.__init__() got an unexpected keyword argument 'type'


100%|██████████| 65.9M/65.9M [00:02<00:00, 24.6MB/s]
  0%|          | 0.00/80.7M [00:00<?, ?B/s]

Upload successful: pair_140.npz (66MB)
Starting upload for file pair_204.npz
Error while trying to load upload info: ApiStartBlobUploadRequest.__init__() got an unexpected keyword argument 'type'


...

100%|██████████| 97.8M/97.8M [00:03<00:00, 30.6MB/s]
  0%|          | 0.00/88.2M [00:00<?, ?B/s]

Upload successful: pair_26.npz (98MB)
Starting upload for file pair_142.npz


100%|██████████| 88.2M/88.2M [00:03<00:00, 29.3MB/s]


Upload successful: pair_142.npz (88MB)
Your private Dataset is being created. Please check progress at https://www.kaggle.com/datasets/cvbnqq/kitti360-preprocessed-pairs
Output: None
Errors: None


: 